In [ ]:
#@markdown ####Latest Update: 12-5-2023 4:19 PM UTC - Comfy Colab v1.1
#@markdown \
#@markdown ## Give your worker a name
#@markdown ##### (If you have used this before, please re-use your worker name)
# The horde url
horde_url = "https://aihorde.net" 
worker_name = "Give your worker a cool name" #@param {type:"string"}

if worker_name == "" or worker_name == "Give me a cool worker name":
  print("Please enter a new worker name and resubmit")
#@markdown \
#@markdown ## Enter your API key for the horde
#@markdown ##### (If you havent got one, go to https://aihorde.net/register)
api_key = "" #@param {type:"string"}

if api_key == "":
  print("Please enter an API key and resubmit")
#@markdown \


#@markdown ### Choose a mode
mode = "Simple" #@param ["Simple", "Pick My Model", "Inpainter", "Interrogation"]
    
#@markdown *Simple Mode = No input required, just click and go* \
#@markdown *Inpainting Mode = Runs the inpainter model and stable_diffusion (limited to 768x768, no post-processors)* \
#@markdown *Interrogation Mode = Fulfils requests for captions, tags and nsfw* \
#@markdown *Pick A Model Mode = Runs only the model selected below* \
#@markdown - *('top 1' top automatically selects the most popular model)*


if mode == "Pick My Model":
    pick_my_model = "top 1" #@param ["top 1", "3DKX", "526Mix-Animated", "A to Zovya RPG", "ACertainThing", "AIO Pixel Art", "Abyss OrangeMix", "AbyssOrangeMix-AfterDark", "Analog Diffusion", "Analog Madness", "Anime Pencil Diffusion", "Anygen", "Anything Diffusion", "Anything v3", "App Icon Diffusion", "Arcane Diffusion", "Archer Diffusion", "Art Of Mtg", "Asim Simpsons", "Aurora", "BB95 Furry Mix", "BPModel", "Balloon Art", "Beautiful Realistic Asians", "Borderlands", "BubblyDubbly", "CamelliaMix 2.5D", "Char", "CharHelper", "Cheese Daddys Landscape Mix", "ChilloutMix", "ChromaV5", "Classic Animation Diffusion", "Clazy", "Colorful", "Coloring Book", "Comic-Diffusion", "Concept Sheet", "Counterfeit", "Cyberpunk Anime Diffusion", "CyriousMix", "DGSpitzer Art Diffusion", "Dan Mumford Style", "Dark Victorian Diffusion", "Darkest Diffusion", "Deliberate", "Disco Elysium", "DnD Item", "DnD Map Generator", "Double Exposure Diffusion", "DreamLikeSamKuvshinov", "Dreamlike Diffusion", "Dreamlike Photoreal", "Dreamshaper", "DucHaiten", "DucHaiten Classic Anime", "Dungeons and Diffusion", "Dungeons n Waifus", "Edge Of Realism", "Eimis Anime Diffusion", "Elden Ring Diffusion", "Elldreth's Lucid Mix", "Elldreths Retro Mix", "Epic Diffusion", "Eternos", "Ether Real Mix", "ExpMix Line", "Experience", "FKing SciFi", "FaeTastic", "Fantasy Card Diffusion", "Funko Diffusion", "Furry Epoch", "Future Diffusion", "GTA5 Artwork Diffusion", "Galena Redux", "Ghibli Diffusion", "GorynichMix", "Grapefruit Hentai", "Graphic-Art", "GuFeng", "GuoFeng", "Guohua Diffusion", "HASDX", "HRL", "Hassanblend", "Healy's Anime Blend", "Hentai Diffusion", "ICBINP - I Can't Believe It's Not Photography", "Illuminati Diffusion", "Inkpunk Diffusion", "JWST Deep Space Diffusion", "Jim Eidomode", "JoMad Diffusion", "Kenshi", "Knollingcase", "Korestyle", "Laolei New Berry Protogen Mix", "Lawlas's yiff mix", "Liberty", "Lyriel", "Marvel Diffusion", "Mega Merge Diffusion", "MeinaMix", "Microcasing", "Microchars", "Microcritters", "Microscopic", "Microworlds", "Midjourney Diffusion", "Midjourney PaintArt", "Min Illust Background", "Mistoon Amethyst", "ModernArt Diffusion", "Moedel", "MoistMix", "MoonMix Fantasy", "Movie Diffusion", "Neurogen", "NeverEnding Dream", "Nitro Diffusion", "Openniji", "OrbAI", "PFG", "PIXHELL", "PPP", "PRMJ", "PVC", "Papercut Diffusion", "Papercutcraft", "Pastel Mix", "Perfect World", "Poison", "Pokemon3D", "PortraitPlus", "Pretty 2.5D", "Project Unreal Engine 5", "ProtoGen", "Protogen Anime", "Protogen Infinity", "Pulp Vector Art", "RCNZ Dumb Monkey", "RCNZ Gorilla With A Brick", "RPG", "Rachel Walker Watercolors", "Rainbowpatch", "Ranma Diffusion", "RealBiter", "Realism Engine", "Realistic Vision", "Redshift Diffusion", "Rev Animated", "Robo-Diffusion", "Rodent Diffusion", "SD-Silicon", "Samdoesarts Ultmerge", "Sci-Fi Diffusion", "Seek.art MEGA", "Smoke Diffusion", "Something", "Sonic Diffusion", "Spider-Verse Diffusion", "Squishmallow Diffusion", "Supermarionation", "Sygil-Dev Diffusion", "Synthwave", "SynthwavePunk", "T-Shirt Diffusion", "T-Shirt Print Designs", "TrexMix", "Trinart Characters", "Tron Legacy Diffusion", "UMI Olympus", "URPM", "Uhmami", "Ultraskin", "Unstable Ink Dream", "Valorant Diffusion", "Van Gogh Diffusion", "Vector Art", "VinteProtogenMix", "Vintedois Diffusion", "Vivid Watercolors", "Voxel Art Diffusion", "Wavyfusion", "Woop-Woop Photo", "Xynthii-Diffusion", "Yiffy", "Zack3D", "Zeipher Female Model", "Zelda BOTW", "iCoMix", "kurzgesagt", "mo-di-diffusion", "pix2pix", "stable_diffusion", "stable_diffusion_2.1", "trinart", "vectorartz", "waifu_diffusion"]


#@markdown ### Show Logs
Show_Logs = True #@param {type:"boolean"}
#@markdown *If ticked, this will show a log in the output console for each generation.  If unticked, a flashing cursor will appear until an error occurs and the log is in the AI-Horde-Worker/logs folder* 

#@markdown \

#@markdown ## **When finished, press control+f9 to run all the cells, or use the menu at the top (Runtime -> Run all).**
#@markdown ### It will take about 5 minutes for the worker to start processing jobs. Check in to make sure no errors are being encountered every so often.
if mode == "Inpainter":
  allow_painting = True
  dynamic_models = False
  max_power = 16
  models_to_load = ["stable_diffusion_inpainting"]
  models_to_skip = ["stable_diffusion_2.1",  "stable_diffusion_2.0"]
  allow_post_processing = False

elif mode == "Pick My Model":
  allow_painting = False
  dynamic_models = False
  max_power = 16
  models_to_load = [f"{pick_my_model}"]
  models_to_skip = ["stable_diffusion_2.1",  "stable_diffusion_2.0"]
  allow_post_processing = False

else:
  allow_painting = False
  dynamic_models = False
  max_power = 16
  models_to_load = ["top 1"]
  models_to_skip = ["stable_diffusion_inpainting", "stable_diffusion_2.1",  "stable_diffusion_2.0"]
  allow_post_processing = False

###  IGNORE FROM HERE UNLESS YOU KNOW WHAT YOU ARE DOING ###
priority_usernames = []
max_threads = 1
nsfw = True
censor_nsfw = False
blacklist = []
censorlist = []
allow_img2img = True
allow_controlnet = False
allow_unsafe_ip = True
number_of_dynamic_models = 0
max_models_to_download = 12
forms = ["caption","nsfw","interrogation","post-process"]

In [ ]:
#@markdown # Download and Install Stable Diffusion for Stable Horde
#@markdown This takes ~5 minutes - please ignore any red errors telling you to restart the runtime after installing a new version of something

%cd /content/
!git clone https://github.com/db0/AI-Horde-Worker.git

%cd /content/AI-Horde-Worker
!pip install -r requirements.txt



In [ ]:
#@markdown # Start Stable Horde worker
#@markdown For maximum runtime, leave this browser tab running and clear the logs every 15-30 mins by hitting the X in the top left corner of the logs (will appear when hovering over the three dots)
import os

%cd /content/AI-Horde-Worker

!rm -rf bridgeData.yaml

from yaml import load, dump
def make_yaml_sublist(list_to_convert: list[str]):
  sublist_yaml = dump(list_to_convert)
  sublist_yaml = sublist_yaml.replace('- ', '  - "')
  sublist_yaml = sublist_yaml.replace('\n', '"\n')
  sublist_yaml = "\n" + sublist_yaml 
  return sublist_yaml

if "api_key" not in locals():
  print()
  print("*" * 80)
  print("ERROR: You haven't set an API key... double check the first cell at the top of the page.")
  print("*" * 80)
  print()

data = f"""horde_url: "{horde_url}"
api_key: "{api_key}"
priority_usernames: []
max_threads: {max_threads}
queue_size: 1
require_upfront_kudos: false
dreamer_name: "{worker_name}"
max_power: {max_power}
nsfw: {nsfw}
censor_nsfw: false
blacklist: {blacklist}
censorlist: {censorlist}
allow_img2img: {allow_img2img}
allow_painting: {allow_painting.__str__().lower()}
allow_unsafe_ip: true
allow_post_processing: {allow_post_processing.__str__().lower()}
allow_controlnet: false 
dynamic_models: false
number_of_dynamic_models: 0
max_models_to_download: 10
stats_output_frequency: 30
cache_home: "./"
always_download: true
temp_dir: "./tmp"
enable_terminal_ui: false
vram_to_leave_free: "40%"
ram_to_leave_free: "80%"
disable_disk_cache: false
models_to_load: {make_yaml_sublist(models_to_load)}
models_to_skip: {make_yaml_sublist(models_to_skip)}
forms:
  - "caption"
  - "nsfw" 
  - "interrogation"
  - "post-process"
"""

toExec = """text_file = open("bridgeData.yaml", "w+"); text_file.write(data); text_file.close()""" 
exec(toExec)

!export LOW_VRAM_MODE=0
if mode == "Interrogation":
  if Show_Logs:
    !python bridge_alchemy.py -y
  else:
    print ("Your worker has started and is running while the cursor below is flashing")
    !python bridge_alchemy.py -y -q

else:
  if Show_Logs:
    !python bridge_stable_diffusion.py -y
  else:
    print ("Your worker has started and is running while the cursor below is flashing")
    !python bridge_stable_diffusion.py -y -q